In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gzip
import os
import h5py

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My\ Drive/Group 4/

/content/drive/.shortcut-targets-by-id/1-A7MSZ6V8eP_xZaDJ_j7yN9ddVL8-jgm/Group 4/eeg_full


Function to list patient names

In [ ]:
def list_patient_names(root):

    patient_list = sorted([item for item in os.listdir(root) 
                           if os.path.isdir(os.path.join(root, item))],
                           key = lambda x: x[4:])
    
    return patient_list

Function to list trial files

In [ ]:
def list_trial_files(root, patient_name):
    
    patient_folder = root + patient_name + "/"
    
    trial_names = sorted([f for f in os.listdir(patient_folder) 
                          if os.path.isfile(os.path.join(patient_folder, f))])
    
    return trial_names

Function to read in patient name, patient type, trial number, stimulus type, sensor name

In [ ]:
def parse_dat(root, patient_name, trial_file):
    
    txtfile = root + patient_name + "/" + trial_file
    
    print(txtfile)
    
    # Read in the entire data file
    lines = []

    with gzip.open(txtfile,'rt') as f:
        for line in f:
            lines.append(line)
            
    # Get the patient name
    patient_name = lines[0].split(' ')[1][:-4]
    
    # Get patient type (alcoholic or control)
    patient_type = patient_name[3]
    
    # Get trial number
    trial_num = trial_file[-6:-3]
    
    # Get stimulus type
    stim_type = lines[3].split(' ')[1] + ' ' + lines[3].split(' ')[2]
    
    # Get sensor names
    readings = lines[4:]
    headers = [r for r in readings if r[0]=='#']
    sensor_names = [h.split(' ')[1] for h in headers]
        
    # Get sensor readings for all 64 detectors for all 256 timestamps
    data = [r for r in readings if r[0]!='#']
    
    data_mat = np.zeros((64, 256))
    
    for i in range(64):
        for j in range(256):
            data_mat[i, j] = float(data[i*256+j].split(' ')[3][:-1])
            
    return patient_name, patient_type, trial_num, stim_type, sensor_names, data_mat

Draft - making a dataframe for all patient cases and storing the data in a parseable format

In [ ]:
root = os.getcwd()
data_path = root + 'eeg_full/'

In [ ]:
filename = data_path+'err_files.txt'

In [ ]:
with open(filename) as file:
    err_files = [line.rstrip() for line in file]

In [ ]:
err_files

['co2c1000367.rd.004.gz',
 'co2c1000367.rd.005.gz',
 'co2c1000367.rd.006.gz',
 'co2c1000367.rd.023.gz',
 'co2c1000367.rd.029.gz',
 'co2c1000367.rd.037.gz',
 'co2c1000367.rd.042.gz',
 'co2c1000367.rd.053.gz',
 'co2c1000367.rd.054.gz',
 'co2c1000367.rd.065.gz',
 'co2c1000367.rd.089.gz',
 'co2c1000367.rd.090.gz',
 'co2c1000367.rd.105.gz',
 'co2c1000367.rd.113.gz',
 'co2c1000367.rd.114.gz',
 'co2c1000367.rd.116.gz',
 'co2c1000367.rd.117.gz',
 'co2c0000341.rd.017.gz',
 'co2c0000341.rd.037.gz',
 'co2c0000341.rd.119.gz',
 'co2c0000342.rd.017.gz',
 'co2c0000345.rd.055.gz',
 'co2c0000348.rd.109.gz',
 'co2c0000354.rd.041.gz',
 'co2a0000364.rd.011.gz',
 'co2a0000365.rd.003.gz',
 'co2a0000365.rd.075.gz',
 'co2a0000365.rd.097.gz',
 'co2a0000369.rd.079.gz',
 'co2a0000370.rd.017.gz',
 'co2c0000379.rd.009.gz',
 'co2c0000379.rd.113.gz',
 'co2a0000381.rd.061.gz',
 'co2a0000381.rd.071.gz',
 'co2a0000381.rd.091.gz',
 'co2c0000381.rd.119.gz',
 'co2a0000382.rd.025.gz',
 'co2c0000383.rd.097.gz',
 'co2a000038

In [ ]:
patient_list = list_patient_names(root)
patient_list

['co2c0000337',
 'co2c0000338',
 'co2c0000339',
 'co2c0000340',
 'co2c0000341',
 'co2c0000342',
 'co2c0000344',
 'co2c0000345',
 'co2c0000346',
 'co2c0000347',
 'co2c0000348',
 'co2c0000351',
 'co2c0000352',
 'co2c0000354',
 'co2c0000355',
 'co2c0000356',
 'co2c0000357',
 'co2c0000359',
 'co2c0000362',
 'co2c0000363',
 'co2c0000364',
 'co2a0000364',
 'co2a0000365',
 'co2c0000367',
 'co2a0000368',
 'co2a0000369',
 'co2a0000370',
 'co2c0000370',
 'co2a0000371',
 'co2c0000371',
 'co2a0000372',
 'co2c0000373',
 'co2c0000374',
 'co2a0000375',
 'co2a0000377',
 'co2a0000378',
 'co2c0000378',
 'co2a0000379',
 'co2c0000379',
 'co2a0000380',
 'co2a0000381',
 'co2c0000381',
 'co2a0000382',
 'co2c0000382',
 'co2c0000383',
 'co2a0000384',
 'co2c0000384',
 'co2a0000385',
 'co2a0000386',
 'co2a0000387',
 'co2c0000387',
 'co2a0000388',
 'co2c0000388',
 'co2c0000389',
 'co2c0000390',
 'co2a0000390',
 'co2c0000391',
 'co2c0000392',
 'co2a0000392',
 'co2c0000393',
 'co2c0000394',
 'co2a0000394',
 'co2c00

In [ ]:
S1_df = pd.DataFrame(columns = ['Patient', 'Type', 'Trial'])
hf_S1 = h5py.File(root+'S1.h5', 'w')

In [ ]:
S2_match_df = pd.DataFrame(columns = ['Patient', 'Type', 'Trial'])
hf_S2_match = h5py.File(root+'S2_match.h5', 'w')

In [ ]:
S2_nonmatch_df = pd.DataFrame(columns = ['Patient', 'Type', 'Trial'])
hf_S2_nonmatch = h5py.File(root+'S2_nonmatch.h5', 'w')

In [ ]:
for p in patient_list:
    
    trial_names = list_trial_files(root, p)
    
    for t in trial_names:
        
        if t in err_files:
            
            pass
        
        else:
            try:
            
                patient_name, patient_type, trial_num, stim_type, sensor_names, dat_mat = parse_dat(root, p, t)

                df_row = {'Patient': patient_name, 'Type': patient_type, 'Trial': trial_num}

                if (stim_type == 'S1 obj'):
                    S1_df = S1_df.append(df_row, ignore_index = True)
                    hf_S1.create_dataset(t[:-3], data = dat_mat)

                elif (stim_type == 'S2 match'):
                    S2_match_df = S2_match_df.append(df_row, ignore_index = True)
                    hf_S2_match.create_dataset(t[:-3], data = dat_mat)

                else:
                    S2_nonmatch_df = S2_nonmatch_df.append(df_row, ignore_index = True)
                    hf_S2_nonmatch.create_dataset(t[:-3], data = dat_mat)
            
            except:
                pass

/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000337/co2c0000337.rd.000.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000337/co2c0000337.rd.002.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000337/co2c0000337.rd.016.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000337/co2c0000337.rd.017.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000337/co2c0000337.rd.021.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000337/co2c0000337.rd.023.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000337/co2c0000337.rd.024.gz
/Users/thepoetoftwilight/Documents/UCLA C

/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000337/co2c0000337.rd.098.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000337/co2c0000337.rd.099.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000337/co2c0000337.rd.101.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000337/co2c0000337.rd.102.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000337/co2c0000337.rd.103.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000337/co2c0000337.rd.104.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000337/co2c0000337.rd.105.gz
/Users/thepoetoftwilight/Documents/UCLA C

/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000338/co2c0000338.rd.051.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000338/co2c0000338.rd.052.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000338/co2c0000338.rd.053.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000338/co2c0000338.rd.054.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000338/co2c0000338.rd.055.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000338/co2c0000338.rd.056.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000338/co2c0000338.rd.057.gz
/Users/thepoetoftwilight/Documents/UCLA C

/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000339/co2c0000339.rd.007.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000339/co2c0000339.rd.008.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000339/co2c0000339.rd.012.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000339/co2c0000339.rd.019.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000339/co2c0000339.rd.020.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000339/co2c0000339.rd.022.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000339/co2c0000339.rd.025.gz
/Users/thepoetoftwilight/Documents/UCLA C

/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000339/co2c0000339.rd.103.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000339/co2c0000339.rd.104.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000339/co2c0000339.rd.105.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000339/co2c0000339.rd.106.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000339/co2c0000339.rd.109.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000339/co2c0000339.rd.110.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c0000339/co2c0000339.rd.111.gz
/Users/thepoetoftwilight/Documents/UCLA C

...

/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000460/co3a0000460.rd.025.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000460/co3a0000460.rd.026.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000460/co3a0000460.rd.027.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000460/co3a0000460.rd.028.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000460/co3a0000460.rd.029.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000460/co3a0000460.rd.030.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000460/co3a0000460.rd.031.gz
/Users/thepoetoftwilight/Documents/UCLA C

/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000460/co3a0000460.rd.091.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000460/co3a0000460.rd.092.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000460/co3a0000460.rd.094.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000460/co3a0000460.rd.095.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000460/co3a0000460.rd.096.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000460/co3a0000460.rd.097.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000460/co3a0000460.rd.098.gz
/Users/thepoetoftwilight/Documents/UCLA C

/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000461/co3a0000461.rd.061.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000461/co3a0000461.rd.062.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000461/co3a0000461.rd.063.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000461/co3a0000461.rd.065.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000461/co3a0000461.rd.067.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000461/co3a0000461.rd.069.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co3a0000461/co3a0000461.rd.071.gz
/Users/thepoetoftwilight/Documents/UCLA C

/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c1000367/co2c1000367.rd.035.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c1000367/co2c1000367.rd.036.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c1000367/co2c1000367.rd.038.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c1000367/co2c1000367.rd.039.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c1000367/co2c1000367.rd.040.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c1000367/co2c1000367.rd.041.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c1000367/co2c1000367.rd.043.gz
/Users/thepoetoftwilight/Documents/UCLA C

/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c1000367/co2c1000367.rd.108.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c1000367/co2c1000367.rd.109.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c1000367/co2c1000367.rd.110.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c1000367/co2c1000367.rd.111.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c1000367/co2c1000367.rd.112.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c1000367/co2c1000367.rd.115.gz
/Users/thepoetoftwilight/Documents/UCLA Classes/XIII - Winter 2022/Physics 180M/Final Project/eeg_full/co2c1000367/co2c1000367.rd.118.gz
/Users/thepoetoftwilight/Documents/UCLA C

In [ ]:
S1_df.to_csv(data_path + 'S1_patient_summary.csv', index=None)
hf_S1.close()

In [ ]:
S2_match_df.to_csv(data_path + 'S2_match_patient_summary.csv', index=None)
hf_S2_match.close()

In [ ]:
S2_nonmatch_df.to_csv(data_path + 'S2_nonmatch_patient_summary.csv', index=None)
hf_S2_nonmatch.close()

In [ ]:
textfile = open(data_path+"sensor_names.txt", "w")
for element in sensor_names:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
len(S1_df) + len(S2_match_df) + len(S2_nonmatch_df)

10997